In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
## load data
path = '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/phenotype_file/release_36K/'
SUBJ_drop = pd.read_excel(path + 'ng00067_subject_droplist_2022.08.18.xlsx', engine = 'openpyxl') ## Note: This one is for 17k

manifest = pd.read_csv(path + "SampleManifest_DS_2022.08.18_ALL.txt", sep='\t') ## the one with mapping info
pheno_sampleID = pd.read_csv(path + "pheno_merge_sampleID.tsv", sep='\t')
pheno_merge = pd.read_csv(path + 'phenotype_merged.tsv', sep = '\t')
king = np.loadtxt("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/phenotype_file/release_36K/unrelated_samples.txt", dtype=str) ## The non-related individual's sample ID calculate from Kinship only have sample ID

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [85]:
pheno_merge.shape

(38163, 8)

In [34]:
print(pheno_sampleID.shape)
print(king.shape)

(36246, 17)
(33572,)


### Remove the SUBJID that is in SUBJ_drop

In [37]:
## the only two rows in manifest file that got removed because appearing in SUBJ_drop
print(manifest[(manifest.SUBJID.isin(pheno_merge.SUBJID)) & (manifest.SUBJID.isin(SUBJ_drop.SUBJID))])

manifest = manifest[(manifest.SUBJID.isin(pheno_merge.SUBJID)) & (~manifest.SUBJID.isin(SUBJ_drop.SUBJID))]

print(manifest.shape)

               SUBJID                         SampleID BODY_SITE ANALYTE_TYPE  \
12879  A-MIA-UM000315  A-MIA-UM000315-UNK-MIA-19961724   Unknown          DNA   
12954  A-MIA-UM001976  A-MIA-UM001976-UNK-MIA-20010205   Unknown          DNA   

      Sequencing_Center       Sequencing_Platform SAMPLE_USE  \
12879           UM_HIHG  Illumina_HiSeq_2000/2500        WES   
12954           UM_HIHG  Illumina_HiSeq_2000/2500        WES   

       Technical_Replicate Study_DSS Sample_Set  
12879                    0  sa000006   snd10006  
12954                    0  sa000006   snd10006  
(40644, 10)


In [42]:
manifest[(manifest.SUBJID.duplicated(keep=False))]

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
4,A-ACT-AC000014,A-ACT-AC000014-BL-UPN-23967,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
13,A-ACT-AC000034,A-ACT-AC000034-BL-UPN-15865,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
26,A-ACT-AC000057,A-ACT-AC000057-BL-UPN-14863,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
31,A-ACT-AC000072,A-ACT-AC000072-BL-UPN-7532,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
38,A-ACT-AC000088,A-ACT-AC000088-BL-UPN-13977,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
...,...,...,...,...,...,...,...,...,...,...
56858,A-LOAD-LD004371,A-LOAD-LD004371-BL-NCR-04AD7188u2-H07,Blood,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031
56859,A-LOAD-LD004371,A-LOAD-LD004371-BL-NCR-04AD7188u3-H07,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,1,sa000001,snd10031
56860,A-NCRD-NC005389,A-NCRD-NC005389-BR-NCR-91M0544u1-B01,Brain,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031
56861,A-NCRD-NC005389,A-NCRD-NC005389-BR-NCR-91M0544u2-H08,Brain,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031


In [43]:
manifest[(manifest.SUBJID.duplicated(keep=False))].value_counts('SUBJID') ## total row: 8704

SUBJID
A-CUHS-CU001166    13
A-CUHS-CU001163    13
A-CUHS-CU001167    12
A-CUHS-CU009833    10
A-NCRD-NC005389    10
                   ..
A-MIA-UM009708      2
A-MIA-UM009707      2
A-MIA-UM009706      2
A-MIA-UM009705      2
A-ACT-AC000014      2
Length: 4306, dtype: int64

### Take the intersection with King

In [46]:
from_king = manifest[(manifest.SUBJID.duplicated(keep=False))& manifest.SampleID.isin(king)] ## 3987 of the 4306 duplicated SUBJID were found in King

In [47]:
from_king.shape

(3987, 10)

In [57]:
manifest[manifest.SampleID.isin(king)]

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
20504,A-LOAD-LD004558,A-LOAD-LD004558-BL-NCR-04AD6349,Blood,DNA,WashU,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20509,A-LOAD-LD006391,A-LOAD-LD006391-BR-NCR-03AD5040,Brain,DNA,WashU,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20510,A-LOAD-LD006392,A-LOAD-LD006392-BR-NCR-03AD5039,Brain,DNA,WashU,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20511,A-LOAD-LD006393,A-LOAD-LD006393-BR-NCR-05AD8062,Brain,DNA,WashU,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20516,A-MIA-UM000311,A-MIA-UM000311-BR-MIA-201307291,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
...,...,...,...,...,...,...,...,...,...,...
56840,A-ADC-AD013032,A-ADC-AD013032-BL-NCR-15AD81841,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56841,A-ADC-AD015544,A-ADC-AD015544-BL-NCR-16AD86202,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56842,A-ADC-AD015476,A-ADC-AD015476-BL-NCR-13AD59678,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56843,A-ADC-AD015266,A-ADC-AD015266-BR-NCR-14AD74741,Brain,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031


In [60]:
manifest[(manifest.SUBJID.duplicated(keep=False)) & ~manifest.SampleID.isin(king)].value_counts('SUBJID')

SUBJID
A-CUHS-CU001166    13
A-CUHS-CU001163    12
A-CUHS-CU001167    12
A-CUHS-CU009833     9
A-NCRD-NC005389     9
                   ..
A-MIA-UM009592      1
A-MIA-UM009590      1
A-MIA-UM009589      1
A-MIA-UM009588      1
A-ACT-AC000014      1
Length: 4306, dtype: int64

## deal with the duplicates outside of the intersection (Shouldn't be any duplicate in the intersection)

In [126]:
dup_notfindinking = manifest[(manifest.SUBJID.duplicated(keep=False))& ~(manifest.SampleID.isin(king))]

In [127]:
dup_notfindinking = dup_notfindinking.groupby(['SUBJID']).filter(lambda x: len(x) > 1)

In [125]:
## extract the WGS one if theres one
print(dup_notfindinking.value_counts('SAMPLE_USE'))
print()
len(dup_notfindinking[dup_notfindinking.SAMPLE_USE == 'WGS'].groupby(['SUBJID']).filter(lambda x: len(x) > 1).SUBJID.unique())

SAMPLE_USE
WES    518
WGS    235
dtype: int64



30

In [108]:
dup_notfindinking[dup_notfindinking.SAMPLE_USE == 'WGS'].groupby(['SUBJID']).filter(lambda x: len(x) > 1)

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
20515,A-MIA-UM000037,A-MIA-UM000037-BR-MIA-19890652,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20521,A-MIA-UM000463,A-MIA-UM000463-BR-MIA-201307288,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20522,A-MIA-UM000474,A-MIA-UM000474-BR-MIA-201307282,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20523,A-MIA-UM000477,A-MIA-UM000477-BR-MIA-201307292,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
20527,A-MIA-UM001127,A-MIA-UM001127-BL-MIA-200426255,Blood,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
...,...,...,...,...,...,...,...,...,...,...
56858,A-LOAD-LD004371,A-LOAD-LD004371-BL-NCR-04AD7188u2-H07,Blood,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031
56859,A-LOAD-LD004371,A-LOAD-LD004371-BL-NCR-04AD7188u3-H07,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,1,sa000001,snd10031
56860,A-NCRD-NC005389,A-NCRD-NC005389-BR-NCR-91M0544u1-B01,Brain,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031
56861,A-NCRD-NC005389,A-NCRD-NC005389-BR-NCR-91M0544u2-H08,Brain,DNA,USUHS,Illumina_HiSeqX,WGS,1,sa000001,snd10031


In [110]:
manifest[manifest.SUBJID == 'A-MIA-UM000037']

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
12868,A-MIA-UM000037,A-MIA-UM000037-UNK-MIA-19890652,Unknown,DNA,UM_HIHG,Illumina_HiSeq_2000/2500,WES,0,sa000006,snd10006
20515,A-MIA-UM000037,A-MIA-UM000037-BR-MIA-19890652,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
37399,A-MIA-UM000037,A-MIA-UM000037-BR-MIA-201307277,Brain,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10020


In [111]:
manifest[manifest.SUBJID == 'A-MIA-UM000463']

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
5969,A-MIA-UM000463,A-ADC-AD002288-BR-NCR-08AD10497,Brain,DNA,Broad,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
20521,A-MIA-UM000463,A-MIA-UM000463-BR-MIA-201307288,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
35781,A-MIA-UM000463,A-MIA-UM000463-BR-MIA-201307288uf,Brain,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10020


In [113]:
# Only keeping the first row
WGS = dup_notfindinking[dup_notfindinking.SAMPLE_USE == 'WGS'].drop_duplicates(subset= 'SUBJID')
WES = dup_notfindinking[(dup_notfindinking.SAMPLE_USE == 'WES') & (dup_notfindinking.SUBJID.isin(WGS.SUBJID))].drop_duplicates(subset= 'SUBJID')

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
0,A-ACT-AC000004,A-ACT-AC000004-BL-UPN-15872,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
1,A-ACT-AC000007,A-ACT-AC000007-BL-UPN-6888,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
2,A-ACT-AC000008,A-ACT-AC000008-BL-UPN-8307,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
3,A-ACT-AC000010,A-ACT-AC000010-BL-UWA-15286,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
5,A-ACT-AC000015,A-ACT-AC000015-BL-UPN-39345,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
...,...,...,...,...,...,...,...,...,...,...
56840,A-ADC-AD013032,A-ADC-AD013032-BL-NCR-15AD81841,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56841,A-ADC-AD015544,A-ADC-AD015544-BL-NCR-16AD86202,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56842,A-ADC-AD015476,A-ADC-AD015476-BL-NCR-13AD59678,Blood,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031
56843,A-ADC-AD015266,A-ADC-AD015266-BR-NCR-14AD74741,Brain,DNA,USUHS,Illumina_NovaSeq,WGS,0,sa000001,snd10031


In [138]:
manifest_merge = pd.concat([manifest.drop_duplicates(subset='SUBJID',keep = False),from_king, WGS, WES])

In [139]:
pheno_w_sampleID = pheno_merge.merge(manifest_merge)

In [140]:
pheno_w_sampleID

,SUBJID,Sex,Age,Age_baseline,APOE_reported,Race,Diagnosis,source,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
0,ADNI_002_S_0413,1,87,76,33,5,0,ADNI,ADNI_002_S_0413,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
1,ADNI_002_S_0685,1,90,89,33,5,0,ADNI,ADNI_002_S_0685,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
2,ADNI_002_S_0729,1,68,65,34,5,1,ADNI,ADNI_002_S_0729,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
3,ADNI_002_S_1155,0,69,57,33,5,0,ADNI,ADNI_002_S_1155,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
4,ADNI_002_S_1261,1,82,71,33,5,0,ADNI,ADNI_002_S_1261,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36205,A-CUHS-CU000563,0,74,77,34,6,1,case_control,A-CUHS-CU000563-BL-COL-47451BL1,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36206,A-CUHS-CU000387,1,87,71,23,6,0,case_control,A-CUHS-CU000387-BL-COL-38200BL2,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36207,A-CUHS-CU000194,1,73,65,33,6,0,case_control,A-CUHS-CU000194-BL-COL-38470BL1,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36208,A-CUHS-CU000169,1,87,84,33,5,0,case_control,A-CUHS-CU000169-BL-COL-57413BL1dew,Blood,DNA,WashU,Illumina_HiSeqX,WGS,0,sa000001,snd10001


In [141]:
pheno_w_sampleID.SAMPLE_USE.value_counts()

WGS    21608
WES    14602
Name: SAMPLE_USE, dtype: int64

In [142]:
pheno_w_sampleID.to_csv(path+'pheno_merge_sampleID_king.tsv', sep = '\t', index = False)

,SUBJID,Sex,Age,Age_baseline,APOE_reported,Race,Diagnosis,source,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
0,ADNI_002_S_0413,1,87,76,33,5,0,ADNI,ADNI_002_S_0413,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
1,ADNI_002_S_0685,1,90,89,33,5,0,ADNI,ADNI_002_S_0685,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
2,ADNI_002_S_0729,1,68,65,34,5,1,ADNI,ADNI_002_S_0729,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
3,ADNI_002_S_1155,0,69,57,33,5,0,ADNI,ADNI_002_S_1155,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
4,ADNI_002_S_1261,1,82,71,33,5,0,ADNI,ADNI_002_S_1261,Blood,DNA,Illumina,Illumina_HiSeq_2000,WGS,0,sa000002,snd10002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36241,A-CUHS-CU000563,0,74,77,34,6,1,case_control,A-CUHS-CU000563-BL-COL-47451BL1,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36242,A-CUHS-CU000387,1,87,71,23,6,0,case_control,A-CUHS-CU000387-BL-COL-38200BL2,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36243,A-CUHS-CU000194,1,73,65,33,6,0,case_control,A-CUHS-CU000194-BL-COL-38470BL1,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36244,A-CUHS-CU000169,1,87,84,33,5,0,case_control,A-CUHS-CU000169-BL-COL-57413BL1dew,Blood,DNA,WashU,Illumina_HiSeqX,WGS,0,sa000001,snd10001


In [146]:
pheno_w_sampleID[~pheno_w_sampleID.SUBJID.isin(pheno_sampleID.SUBJID)]

,SUBJID,Sex,Age,Age_baseline,APOE_reported,Race,Diagnosis,source,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set


In [147]:
pheno_w_sampleID[~pheno_w_sampleID.SampleID.isin(pheno_sampleID.SampleID)]

,SUBJID,Sex,Age,Age_baseline,APOE_reported,Race,Diagnosis,source,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
1595,A-CUHS-CU001167,1,70,65,33,6,0,Family_based,A-CUHS-CU001167-BL-COL-32167BL1,Blood,DNA,Baylor,Illumina_HiSeq_2000,WGS,1,sa000001,snd10000
1655,A-CUHS-CU006133,1,58,58,44,6,0,Family_based,A-CUHS-CU006133-BL-COL-31008BL1,Blood,DNA,WashU,Illumina_HiSeqX,WGS,0,sa000001,snd10001
1700,A-LOAD-LD004371,1,53,53,34,5,0,Family_based,A-LOAD-LD004371-BL-NCR-04AD7188fa,Blood,DNA,Baylor,Illumina_HiSeqX,WGS,1,sa000001,snd10001
1743,A-MIA-UM000486,0,67,67,33,5,0,Family_based,A-MIA-UM000486-UNK-MIA-19950416,Unknown,DNA,UM_HIHG,Illumina_HiSeq_2000/2500,WES,0,sa000006,snd10006
1758,A-MIA-UM001826,1,85,85,33,5,0,Family_based,A-MIA-UM001826-UNK-MIA-200808817,Unknown,DNA,UM_HIHG,Illumina_HiSeq_2000/2500,WES,0,sa000006,snd10006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36165,A-MIA-UM000336,1,72,72,34,5,1,case_control,A-ADC-AD002827-BR-NCR-08AD10512,Brain,DNA,Broad,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36167,A-MIA-UM000329,0,73,73,34,5,1,case_control,A-NCRD-NC005342-BL-NCR-91M0044,Blood,DNA,WashU,Illumina_HiSeq_2000,WES,0,sa000001,snd10000
36169,A-MIA-UM000037,1,74,87,34,5,1,case_control,A-MIA-UM000037-BR-MIA-19890652,Brain,DNA,Broad,Illumina_HiSeq_2000,WGS,0,sa000001,snd10000
36170,A-MIA-UM000037,1,74,87,34,5,1,case_control,A-MIA-UM000037-UNK-MIA-19890652,Unknown,DNA,UM_HIHG,Illumina_HiSeq_2000/2500,WES,0,sa000006,snd10006
